In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from loaders.loaders import *
lc_quad_train = load_on_path(ds_lc_quad_train_cleaned)

In [4]:
from query.query_functions2 import (
    run_sparql_query_paged, run_sparql_query,
    get_result_from_triples, count_from_triples
)

In [144]:
import itertools
from copy import copy


def split_graph(graph):
    possible = list()
    for i in range(len(graph) - 1):
        combinations = list(itertools.combinations(graph, i+1))
        possible.extend(combinations)
    return possible

def get_missing(selection, alltriples):
    remaining = copy(alltriples)
    exclude_index = list()

    for triple in selection:
        for ind, rem in enumerate(remaining):
            cool = True
            for i in range(len(triple)):
                if triple[i] != rem[i]:
                    cool = False
                    break
            if cool:
                exclude_index.append(ind)
                break

    final_list = [row for i, row in enumerate(remaining) if i not in exclude_index]
    return final_list


In [145]:
def make_triplet(triple):
    query_triplet = []
    for item in triple:
        if item.startswith("?"):
            query_triplet.append(item)
        elif item.startswith("http"):
            query_triplet.append(f"<{item}>")
        else:
            query_triplet.append('"' + item + '"')
    return " ".join(query_triplet)


def clear_query(query):
    res = list()
    for item in query:
        if "uri" not in item:
            continue
        res.append(item["uri"]["value"])
    return res

def same_result(res, expected):
    remaining_a = list()
    remaining_b = list()
    for a in res:
        if a not in expected:
            remaining_a.append(a)

    for b in expected:
        if b not in res:
            remaining_b.append(b)

    return remaining_a + remaining_b

In [146]:
from collections import Counter, defaultdict
optional_rows = set()
optional_query_part = defaultdict(list)


for i, record in enumerate(lc_quad_train):
    # all_count = count_from_triples(record["graph"])
    triples_str = " . ".join([" ".join(item) for item in record['graph']])
    #query_command = f"SELECT count(?uri) WHERE {{ {triples_str} }}"
    query_command = f"SELECT ?uri WHERE {{ {triples_str} }}"
    #all_count = run_sparql_query(query_command)
    all_nodes = clear_query(run_sparql_query(query_command))

    parts = split_graph(record["graph"])
    for item in parts:
        triples_str = " . ".join([" ".join(item) for item in item])
        #part_command = f"SELECT count(?uri) WHERE {{ {triples_str} }}"
        part_command = f"SELECT ?uri WHERE {{ {triples_str} }}"
        part_nodes = clear_query(run_sparql_query(part_command))
        #part_count = run_sparql_query(query_command)
        # part_count = count_from_triples(item)
        difference = same_result(part_nodes, all_nodes)
        if len(difference) == 0:
            optional_rows.add(i)
            optional_query_part[i].append(get_missing(item, record["graph"]))


In [147]:
len(optional_rows)

131

In [148]:
optional_rows

{1,
 2,
 3,
 5,
 11,
 12,
 15,
 19,
 22,
 23,
 24,
 27,
 30,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 44,
 46,
 48,
 49,
 50,
 51,
 56,
 57,
 58,
 59,
 61,
 63,
 64,
 68,
 69,
 71,
 74,
 76,
 79,
 83,
 84,
 86,
 89,
 90,
 91,
 94,
 98,
 99,
 101,
 103,
 104,
 105,
 107,
 109,
 111,
 113,
 114,
 117,
 120,
 121,
 123,
 124,
 125,
 127,
 131,
 132,
 134,
 137,
 142,
 143,
 145,
 147,
 148,
 149,
 154,
 155,
 156,
 159,
 160,
 161,
 165,
 167,
 168,
 170,
 172,
 175,
 177,
 178,
 179,
 180,
 182,
 184,
 185,
 187,
 188,
 189,
 191,
 193,
 197,
 198,
 199,
 201,
 202,
 203,
 204,
 206,
 207,
 209,
 210,
 212,
 213,
 216,
 221,
 222,
 223,
 224,
 225,
 228,
 230,
 232,
 233,
 234,
 238,
 239,
 242,
 243,
 245,
 248,
 249}

In [149]:
for k,v in optional_query_part.items():
    print("K:", k)
    print(v)

K: 1
[[['?x', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/Book>']]]
K: 2
[[['?x', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/EthnicGroup>']]]
K: 3
[[['?x', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/Band>']]]
K: 5
[[['?uri', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/Bacteria>']]]
K: 11
[[['?x', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/Person>']]]
K: 12
[[['?x', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/Automobile>']]]
K: 15
[[['?x', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/Band>']]]
K: 19
[[['?x', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/Company>']]]
K: 22
[[['?x', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://dbpedia.org/ontology/GrandPrix>']]]
K: 

In [150]:
i = list(optional_rows)[4]
record = lc_quad_train[i]

In [151]:
triples_str = " . ".join([" ".join(item) for item in record['graph']])
query_command = f"SELECT ?uri WHERE {{ {triples_str} }}"
all_nodes = clear_query(run_sparql_query(query_command))

parts = split_graph(record["graph"])
for item in parts:
    part_triples_str = " . ".join([" ".join(elem) for elem in item])
    part_command = f"SELECT ?uri WHERE {{ {part_triples_str} }}"
    part_nodes = clear_query(run_sparql_query(part_command))

    difference = same_result(part_nodes, all_nodes)
    if len(part_nodes) == len(all_nodes):
        print("Len:", item)

    if len(difference) == 0:
        print("Diff:", item)
        print("PART NODES:", part_nodes)
        print("ALL NODES:", all_nodes)
        break

Len: (['?x', '<http://dbpedia.org/ontology/battle>', '<http://dbpedia.org/resource/Operation_Barbarossa>'], ['?x', '<http://dbpedia.org/ontology/relation>', '?uri'])
Diff: (['?x', '<http://dbpedia.org/ontology/battle>', '<http://dbpedia.org/resource/Operation_Barbarossa>'], ['?x', '<http://dbpedia.org/ontology/relation>', '?uri'])
PART NODES: ['http://dbpedia.org/resource/Paul_von_Hindenburg', 'http://dbpedia.org/resource/Albrecht_Gustav_von_Manstein', 'http://dbpedia.org/resource/Edel-Heinrich_Zachariae-Lingenthal', 'http://dbpedia.org/resource/Eduard_von_Lewinski', 'http://dbpedia.org/resource/Heinz_Günther_Guderian', 'http://dbpedia.org/resource/Karl_Ernst_Rahtgens', 'http://dbpedia.org/resource/Wolfgang_von_Kluge', 'http://dbpedia.org/resource/Joachim_von_Ribbentrop', 'http://dbpedia.org/resource/Hans-Karl_Stepp', 'http://dbpedia.org/resource/Kurt_von_der_Chevallerie', 'http://dbpedia.org/resource/Kurt_von_Tippelskirch', 'http://dbpedia.org/resource/Helmut_Friebe', 'http://dbpedia.

In [152]:
all_nodes

['http://dbpedia.org/resource/Paul_von_Hindenburg',
 'http://dbpedia.org/resource/Albrecht_Gustav_von_Manstein',
 'http://dbpedia.org/resource/Edel-Heinrich_Zachariae-Lingenthal',
 'http://dbpedia.org/resource/Eduard_von_Lewinski',
 'http://dbpedia.org/resource/Heinz_Günther_Guderian',
 'http://dbpedia.org/resource/Karl_Ernst_Rahtgens',
 'http://dbpedia.org/resource/Wolfgang_von_Kluge',
 'http://dbpedia.org/resource/Joachim_von_Ribbentrop',
 'http://dbpedia.org/resource/Hans-Karl_Stepp',
 'http://dbpedia.org/resource/Kurt_von_der_Chevallerie',
 'http://dbpedia.org/resource/Kurt_von_Tippelskirch',
 'http://dbpedia.org/resource/Helmut_Friebe',
 'http://dbpedia.org/resource/Gottfried_von_Erdmannsdorff',
 'http://dbpedia.org/resource/Friedrich_Sixt_von_Armin',
 'http://dbpedia.org/resource/Eduard_Zorn',
 'http://dbpedia.org/resource/Heinrich_Himmler',
 'http://dbpedia.org/resource/Johannes_Mayer',
 'http://dbpedia.org/resource/Ferdinand_Sauerbruch',
 'http://dbpedia.org/resource/Johann_Pfl

In [153]:
part_nodes

['http://dbpedia.org/resource/Paul_von_Hindenburg',
 'http://dbpedia.org/resource/Albrecht_Gustav_von_Manstein',
 'http://dbpedia.org/resource/Edel-Heinrich_Zachariae-Lingenthal',
 'http://dbpedia.org/resource/Eduard_von_Lewinski',
 'http://dbpedia.org/resource/Heinz_Günther_Guderian',
 'http://dbpedia.org/resource/Karl_Ernst_Rahtgens',
 'http://dbpedia.org/resource/Wolfgang_von_Kluge',
 'http://dbpedia.org/resource/Joachim_von_Ribbentrop',
 'http://dbpedia.org/resource/Hans-Karl_Stepp',
 'http://dbpedia.org/resource/Kurt_von_der_Chevallerie',
 'http://dbpedia.org/resource/Kurt_von_Tippelskirch',
 'http://dbpedia.org/resource/Helmut_Friebe',
 'http://dbpedia.org/resource/Gottfried_von_Erdmannsdorff',
 'http://dbpedia.org/resource/Friedrich_Sixt_von_Armin',
 'http://dbpedia.org/resource/Eduard_Zorn',
 'http://dbpedia.org/resource/Heinrich_Himmler',
 'http://dbpedia.org/resource/Johannes_Mayer',
 'http://dbpedia.org/resource/Ferdinand_Sauerbruch',
 'http://dbpedia.org/resource/Johann_Pfl

In [154]:
part_command

'SELECT ?uri WHERE { ?x <http://dbpedia.org/ontology/battle> <http://dbpedia.org/resource/Operation_Barbarossa> . ?x <http://dbpedia.org/ontology/relation> ?uri }'

In [155]:
query_command

'SELECT ?uri WHERE { ?x <http://dbpedia.org/ontology/battle> <http://dbpedia.org/resource/Operation_Barbarossa> . ?x <http://dbpedia.org/ontology/relation> ?uri . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Person> }'

In [156]:
optional_query_part[i]

[[['?x',
   '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
   '<http://dbpedia.org/ontology/Person>']]]

In [130]:
get_missing(item, record["graph"])

[['?x',
  '<http://dbpedia.org/ontology/battle>',
  '<http://dbpedia.org/resource/Operation_Barbarossa>'],
 ['?x', '<http://dbpedia.org/ontology/relation>', '?uri'],
 ['?x',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://dbpedia.org/ontology/Person>']]

In [142]:
alltriples = copy(record["graph"])
selection = copy(item)
remaining = copy(alltriples)

exclude_index = list()

for triple in selection:
    for ind, rem in enumerate(remaining):
        cool = True
        for i in range(len(triple)):
            if triple[i] != rem[i]:
                cool = False
                break
        if cool:
            print(ind)
            exclude_index.append(ind)
            break

0
1


In [143]:
final_list = [row for i, row in enumerate(remaining) if i not in exclude_index]
final_list

[['?x',
  '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>',
  '<http://dbpedia.org/ontology/Person>']]